In [207]:
import itertools

def all_forking(base, add_teams=False, **sorted_arg):
    base_len = len(base)

    #checking input
    for i in range(base_len):
        if(len(base[i][0]) != len(base[i][1])):
            raise Exception(f"Base iterable is not correct: element {i} has mismatching components, {base[i][0]} vs {base[i][1]}")

    legs = set()
    for i in range(base_len):
        legs.update(base[i][1])
    legs = sorted(list(legs), **sorted_arg)
    leg_no = len(legs)
    
    base_perm = []
    for i in range(base_len):
        base_perm.append(list(itertools.permutations(base[i][0])))

    result_list = []
    for element in itertools.product(*base_perm):
        res = [''] * leg_no
        for i in range(base_len):
            fork_assign = element[i]
            leg_assign = base[i][1]
            for pos in range(len(leg_assign)):
                res[legs.index(leg_assign[pos])] += fork_assign[pos]
        result_list.append(res)

    result = {"legs": legs, "combinations": result_list}
    if add_teams:
        result["teams"] = [''.join(el) for el in result_list]
    
    return result

In [208]:
def compareStrings(str1, str2, decrease=0):
    if len(str1) != len(str2):
        raise Exception(f"Len of strings is not equals: {str1}({len(str1)}) vs {str2}({len(str2)})")
        
    cnt = 0
    for i in range(len(str1)):
        cnt += (1 - decrease * i) if str1[i] == str2[i] else 0

    return cnt

def compareForks(fork1, fork2, decrease=0, punish_legs=True):
    team1 = ''.join(fork1)
    team2 = ''.join(fork2)
    
    if len(team1) != len(team2):
        raise Exception(f"Len of forkings is not equals: {team1}({len(team1)}) vs {team2}({len(team2)})")

    if len(fork1) != len(fork2):
        raise Exception(f"Len of forkings is not equals: {fork1}({len(fork1)}) vs {fork2}({len(fork2)})")
    
    cnt = 0
    for i in range(len(team1)):
        cnt += (1 - decrease * i) if team1[i] == team2[i] else 0

    if punish_legs:
        for i in range(len(fork1)):
            ln = len(fork1[i])
            if(ln > 2):
                r = compareStrings(fork1[i], fork2[i])
                cnt += (ln / (1 + (ln - r) ** 2)) * (1 / (1 + i ** 2))
        
    return cnt

In [209]:
import numpy as np

def forking_matrix(combinations, decrease=0.04, dist_func=lambda x: x**2):
    comb_no = len(combinations)
    cmatr = np.zeros((comb_no, comb_no))

    for i in range(comb_no):
        for j in range(i, comb_no): # or i+1 if diagonal should be 0
            dist = dist_func(compareForks(combinations[i], combinations [j], decrease))
            cmatr[i, j] = dist
            cmatr[j, i] = dist

    return cmatr

In [210]:
import random

def choose_forking(fork_matrix, teams, mult1, mult2, points_range):
    comb_no = fork_matrix.shape[0]

    avail = list(range(comb_no))
    selected = []

    #adding first random point
    new_point = avail[random.randint(0, len(avail) - 1)]
    selected.append(new_point)
    avail.remove(new_point)
    
    for s in range(1, teams):
        m = 0.0
        for i in range(s+1, teams):
            m += mult2 ** (i - s - 1)
        m *= mult1 ** s
    
        #searching for new point to add
        best_score = 1e12
        new_point = -1
        
        num = len(avail)
        for i in range(num):
            point = avail[i]
            score = 0.0
            #adding distance to already selected points
            for k in range(len(selected)):
                score += fork_matrix[selected[k], point] * (mult1 ** k) * (mult2 ** (s - k - 1))
    
            #adding distance to all other points
            if s != teams - 1:
                #avg_dist = cmatr[point, :].sum() / (num - 1)
                avg_dist = np.mean(np.sort(fork_matrix[point, :])[:int((teams - s - 1)*points_range)])
                score += avg_dist * m
    
            if score < best_score:
                best_score = score
                new_point = point
    
        selected.append(new_point)
        avail.remove(new_point)
        
    return selected

In [211]:
def forking_score(selected, fork_matrix, mult1, mult2):
    teams = len(selected)
    score = 0
    
    for i in range(teams):
        for j in range(i+1, teams):
            score += fork_matrix[selected[i], selected[j]] * (mult1 ** i) * (mult2 ** (j - i - 1))
    
    return score

In [212]:
def output_file(selected, forkings, file_name, header, number_offset, output_type):
    with open(file_name, output_type) as my_file:
        my_file.write(header + "\n")
        for i in range(len(selected)):
            opt = forkings["combinations"][selected[i]]
            for j in range(len(opt)):
                my_file.write("{}.{}: {}{}\n".format(number_offset+i+1, j+1, j+1, opt[j].upper()))



In [213]:
from tqdm import tqdm

def generate_forkings(base, teams, file_name, header, number_offset=0, iter=10, decrease=0.04, mult1 = 0.97, mult2 = 0.95, dist_func=lambda x: x**4, points_range=1.1, output_type="w", **sorted_arg):
    af = all_forking(base, **sorted_arg)
    fm = forking_matrix(af["combinations"], decrease, dist_func)
    score = 1e20
    best = []

    for _ in tqdm(range(iter)):
        selected = choose_forking(fm, teams, mult1, mult2, points_range)
        new_score = forking_score(selected, fm, mult1, mult2)
        if new_score < score:
            best = selected.copy()
            score = new_score 
        
    output_file(best, af, file_name, header, number_offset, output_type)

In [214]:
baseM = ((('a', 'b', 'c'), (1, 2, 3)), (('a', 'b', 'c'), (1, 2, 3)), (('a', 'b', 'c'), (1, 2, 3)), (('a', 'b'), (1, 2)), (('a', 'b'), (1, 2)))
baseW = ((('a', 'b', 'c'), (1, 2, 3)), (('a', 'b', 'c'), (1, 2, 3)), (('a', 'b'), (1, 2)), (('a', 'b'), (1, 2)))

generate_forkings(baseM, 60, "EOC2024_Relay-test.txt", "Men", number_offset=0, iter=20)
generate_forkings(baseW, 45, "EOC2024_Relay-test.txt", "Women", number_offset=100, iter=150, output_type="a")


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:27<00:00,  5.42it/s]
